In [1]:
from datetime import datetime
print(f'Päivitetty {datetime.now().date()} / Aki Taanila')

Päivitetty 2025-11-16 / Aki Taanila


# Datan analysointia helpottavia funktioita

Oletus: **ana.py** on tallennettu samaan hakemistoon kuin tämä muistio **ana.ipynb**. 

Oletus on täytetty, jos lataat omalle koneelle hakemiston (repository) https://github.com/taanila/ana. Jos et ole aikaisemmin ladannut hakemistoa githubista, niin lue [latausohje](https://tilastoapu.wordpress.com/2018/11/27/github-esimerkit-omalle-koneelle/).

Tässä muistiossa on esimerkkejä laatimani **ana.py** moduulin funktioiden käytöstä. Funktioiden avulla voit laskea keskeisimmät kuvailevan ja selittävän analytiikan tulokset nopeasti. Toiminnaltaan laajin funktio **ana_excel** laskee ja kirjoittaa Excel-tiedostoon omille taulukkovälilehdilleen siististi muotoiltuna
- kategoristen muuttujien frekvenssitaulukot
- kaikki mahdolliset kahden kategorisen muuttujan ristiintaulukoinnit ja khiin neliö -testit
- dikotomisten muuttujien yhteenvedon
- kaikkien kategoristen muuttujien ristiintaulukoinnit dikotomisten muuttujien kanssa
- kvantitatiivisten muuttujien tilastolliset tunnusluvut, kvantitatiivisten muuttujien tilastolliset tunnusluvut kategoristen muuttujien mukaan ryhmiteltynä ja kahden ryhmän vertailutestit (t-testit)
- korrelaatiokertoimet ja niiden merkitsevyyden testauksen.

Muut funktiot laskevat yksittäisiä kuvailevan ja selittävän analytiikan tuloksia suoraan jupyter-muistioon.

In [2]:
# Tarvittavat tuonnit:
import pandas as pd
import ana
#from xlwings import load

## Datan lataaminen

Datan voit ladata esimerkiksi paikalliselta koneelta, netistä tai auki olevasta Excelistä. Seuraavassa koodisolussa on kolme vaihtoehtoista tapaa ladata Excel-muotoinen tiedosto:
- ensimmäisessä vaihtoehdossa oletetaan että data on tallennettu samaan kansioon tämän muistion kanssa
- toisessa vaihtoehdossa data ladataan netistä
- kolmannessa vaihtoehdossa oletetaan että data on aktiivisena auki olevassa Excelissä. Tällöin ensimmäisen koodisolun komento `from xlwings import load` täytyy aktivoida poistamalla risuaita # -merkki komennon edestä).

Datan lataamisen jälkeen voit tutustua dataan esimerkiksi komennoilla `ana.df`, `ana.df.columns` ja `ana.df.info()`.

In [3]:
ana.df = pd.read_excel('data1.xlsx')
#ana.df = pd.read_excel('https://taanila.fi/data1.xlsx')
#ana.df = load()
ana.df.columns

Index(['nro', 'sukup', 'ikä', 'perhe', 'koulutus', 'palveluv', 'palkka',
       'johto', 'työtov', 'työymp', 'palkkat', 'työteht', 'työterv', 'lomaosa',
       'kuntosa', 'hieroja'],
      dtype='object')

## Alkutoimet

Listaa kategoriset, kvantitatiiviset ja dikotomiset muuttujat **ana.kategoriset**, **ana.kvantit** ja **ana.dikotomiset** listoina. Jos jokin muuttujatyyppi puuttuu datasta, niin jätä tyhjä lista (pelkät hakasulut). Voit lisätä saman muuttujan useampaan kuin yhteen listaan (esimerkiksi asteikolla mitatuille mielipiteille voit haluta käyttää sekä kategoristen että kvantitatiivisten muuttujien menetelmiä).

Jos data on numeromuotoista ja haluat antaa numeroille tekstimuotoiset vastineet, niin määrittele ne **ana.listat** sanakirjana. Tekstimuotoiset arvot täytyy listata vastaavien numeroarvojen mukaisessa numerojärjestyksessä. Jos et määrittele tekstimuotoisia vastineita, niin jätä kuitenkin tyhjä sanakirja (pelkät aaltosulut).

Usein on tarpeen tehdä muutoksia ja täydennyksiä dataan. Katso https://github.com/taanila/data/blob/main/muunna.ipynb

In [4]:
ana.kategoriset = ['sukup', 'perhe', 'koulutus', 'johto', 'työtov', 'työymp', 'palkkat', 'työteht']
ana.kvantit = ['ikä', 'palveluv', 'palkka', 'johto', 'työtov', 'työymp', 'palkkat', 'työteht']
ana.dikotomiset = ['työterv', 'lomaosa', 'kuntosa', 'hieroja']

# toiston välttämiseksi:
tyytyväisyys = ['Erittäin tyytymätön', 'Tyytymätön','Ei tyytymätön eikä tyytyväinen',
                'Tyytyväinen', 'Erittäin tyytyväinen']

ana.listat = {'sukup':['Mies', 'Nainen'],
         'perhe':['Perheetön', 'Perheellinen'],
         'koulutus':['Peruskoulu', '2. aste', 'Korkeakoulu', 'Ylempi korkeakoulu'],
         'johto':tyytyväisyys,
         'työtov':tyytyväisyys,
         'työymp':tyytyväisyys,
         'palkkat':tyytyväisyys,
         'työteht':tyytyväisyys}

## Kategoristen muuttujien arvot

Kannattaa aina tarkistaa kategoristen ja dikotomisten muuttujien arvot.

In [5]:
ana.arvot()

sukup [1 2]
perhe [1 2]
koulutus [ 1.  2.  3.  4. nan]
johto [1 2 3 4 5]
työtov [ 2.  3.  4.  5. nan]
työymp [1 2 3 4 5]
palkkat [1 2 3 4 5]
työteht [1 2 3 4 5]
työterv [ 1. nan]
lomaosa [ 1. nan]
kuntosa [ 1. nan]
hieroja [ 1. nan]


## Tulokset Exceliin

Funktio **ana_excel** kirjoittaa Excel-tiedostoon keskeiset kuvailevan ja selittävän analytiikan tulkset siistissä muodossa.

Valitse nimi Excel-tiedostolle harkiten: jos hakemistossa on saman niminen Excel-tiedosto, niin se menetetään!

Funktion suoritus kestää hetken aikaa. Odota rauhassa ja avaa sen jälkeen Excel-tiedosto.

In [6]:
ana.ana_excel('tulokset.xlsx')

Tulokset löytyvät Excel-tiedostosta tulokset.xlsx.


## Frekvenssitaulukko

Funktio **frekv** laskee frekvenssitaulukon yksittäiselle muuttujalle. 

Anna lähtötietona yksittäisen kategorisen muuttujan nimi.

In [7]:
ana.frekv('johto')

,f,%
Erittäin tyytymätön,7,8.5 %
Tyytymätön,16,19.5 %
Ei tyytymätön eikä tyytyväinen,30,36.6 %
Tyytyväinen,23,28.0 %
Erittäin tyytyväinen,6,7.3 %
Yhteensä,82,100.0 %


## Dikotomisten muuttujien yhteenveto

Funktio **dikot** laskee dikotomisten muuttujien yhteenvedon. 

In [8]:
ana.dikot()

n = 82


,f,% vastaajista
työterv,47,57.3 %
hieroja,22,26.8 %
lomaosa,20,24.4 %
kuntosa,9,11.0 %


## Kvantitatiivisten muuttujien tilastolliset tunnusluvut

Funktio **tunnu** laskee kvantitatiivisten muuttujien tilastolliset tunnusluvut.

In [9]:
ana.tunnu()

,ikä,palveluv,palkka,johto,työtov,työymp,palkkat,työteht
Lukumäärä,82.000000,80.000000,82.000000,82.000000,81.000000,82.000000,82.000000,82.000000
Keskiarvo,37.951220,12.175000,2563.878049,3.060976,4.061728,3.219512,2.109756,3.195122
Keskihajonta,9.773866,8.807038,849.350302,1.058155,0.826826,1.154961,1.111179,1.047502
Pienin,20.000000,0.000000,1521.000000,1.000000,2.000000,1.000000,1.000000,1.000000
Alaneljännes,31.000000,3.750000,2027.000000,2.000000,4.000000,3.000000,1.000000,3.000000
Mediaani,37.500000,12.500000,2320.000000,3.000000,4.000000,3.000000,2.000000,3.000000
Yläneljännes,44.000000,18.250000,2808.000000,4.000000,5.000000,4.000000,3.000000,4.000000
Suurin,61.000000,36.000000,6278.000000,5.000000,5.000000,5.000000,5.000000,5.000000


## Kahden muuttujan välinen ristiintaulukointi

- Funktio **risti_lkm** laskee kahden muuttujan välisen ristiintaulukoinnin lukumäärinä. 
- Funktio **risti** laskee kahden muuttujan välisen ristiintaulukoinnin sarakeprosentteina (prosenttia sarakkeen kokonaislukumäärästä).
- Funktio **risti_khi** laskee khiin neliö -testin.

Anna lähtötietoina kahden kategorisen muuttujan nimet.

In [10]:
# lukumäärät
ana.risti_lkm('johto', 'sukup')

,Mies,Nainen,Yhteensä
Erittäin tyytymätön,7,0,7
Tyytymätön,15,1,16
Ei tyytymätön eikä tyytyväinen,23,7,30
Tyytyväinen,15,8,23
Erittäin tyytyväinen,3,3,6
Yhteensä,63,19,82


In [11]:
# sarakeprosentit
ana.risti('johto', 'sukup')

n-arvot: {'Mies': 63, 'Nainen': 19, 'Yhteensä': 82}


,Mies,Nainen,Yhteensä
Erittäin tyytymätön,11.1 %,0.0 %,8.5 %
Tyytymätön,23.8 %,5.3 %,19.5 %
Ei tyytymätön eikä tyytyväinen,36.5 %,36.8 %,36.6 %
Tyytyväinen,23.8 %,42.1 %,28.0 %
Erittäin tyytyväinen,4.8 %,15.8 %,7.3 %


In [12]:
# khiin neliö -testi
ana.risti_khi('johto', 'sukup')

Khiin neliö = 8.85, p-arvo = 0.065, vapausasteet = 4,
alle viiden suuruisia odotettuja frekvenssejä 40.0 %.

Odotetut frekvenssit:


,Mies,Nainen
Erittäin tyytymätön,5.378049,1.621951
Tyytymätön,12.292683,3.707317
Ei tyytymätön eikä tyytyväinen,23.048780,6.951220
Tyytyväinen,17.670732,5.329268
Erittäin tyytyväinen,4.609756,1.390244


## Muuttujan ristiintaulukointi dikotomisten muuttujien kanssa

Funktio **risti_dikot** ristiintaulukoi yksittäisen muuttujan dikotomisten muuttujien kanssa.

Anna lähtötietona yksittäisen kategorisen muuttujan nimi.

In [13]:
ana.risti_dikot('koulutus')

n-arvot: {'Peruskoulu': 27, '2. aste': 30, 'Korkeakoulu': 22, 'Ylempi korkeakoulu': 2}


,Peruskoulu,2. aste,Korkeakoulu,Ylempi korkeakoulu
työterv,70.4 %,60.0 %,36.4 %,50.0 %
lomaosa,22.2 %,16.7 %,36.4 %,50.0 %
kuntosa,0.0 %,16.7 %,13.6 %,50.0 %
hieroja,22.2 %,36.7 %,18.2 %,50.0 %


## Muuttujan ristiintaulukointi kvantitatiivisen muuttujan kanssa

Funktio **risti_tunnu** ristiintaulukoi yksittäisen muuttujan yksittäisen kvantitatiivisen muuttujan kanssa. Jos muuttuja on kaksiarvoinen, niin lisäksi lasketaan kahden riippumattoman otoksen t-testi (olettaen erisuuret varianssit).

Anna lähtötietona kategorisen muuttujan nimi ja kvantitatiivisen muuttujan nimi.

In [14]:
ana.risti_tunnu('sukup', 'palkka')

t = 3.06, p-arvo = 0.003.


,Mies,Nainen
Lukumäärä,63.000000,19.000000
Keskiarvo,2663.746032,2232.736842
Keskihajonta,930.061860,340.780451
Pienin,1521.000000,1715.000000
Alaneljännes,2086.000000,1969.000000
Mediaani,2417.000000,2144.000000
Yläneljännes,2925.000000,2574.000000
Suurin,6278.000000,2729.000000


## Korrelaatiokertoimet

- Funktio **korre** laskee kvantitatiivisten muuttujien väliset Pearsonin korrelaatiokertoimet.
- Funktio **korre_n** laskee korrelaatiokertoimiin liittyvät n-arvot.
- Funktio **korre_p** laskee korrelaatiokertoimiin liittyvät p-arvot.

In [15]:
# korrelaatiokertoimet
ana.korre()

,ikä,palveluv,palkka,johto,työtov,työymp,palkkat,työteht
ikä,1.000000,0.531559,0.296872,0.120856,0.077554,0.101577,0.072114,0.187848
palveluv,0.531559,1.000000,0.243139,0.073422,0.019732,0.126504,0.004420,0.143100
palkka,0.296872,0.243139,1.000000,0.174133,0.117468,0.267615,0.314889,0.307886
johto,0.120856,0.073422,0.174133,1.000000,0.223328,0.584918,0.540228,0.456933
työtov,0.077554,0.019732,0.117468,0.223328,1.000000,0.285940,0.086845,0.308939
työymp,0.101577,0.126504,0.267615,0.584918,0.285940,1.000000,0.538940,0.576431
palkkat,0.072114,0.004420,0.314889,0.540228,0.086845,0.538940,1.000000,0.479884
työteht,0.187848,0.143100,0.307886,0.456933,0.308939,0.576431,0.479884,1.000000


In [16]:
# korrelaatiokertoimiin liittyvät n-arvot
ana.korre_n()

,ikä,palveluv,palkka,johto,työtov,työymp,palkkat,työteht
ikä,NaN,80,82,82,81,82,82,82
palveluv,80,NaN,80,80,79,80,80,80
palkka,82,80,NaN,82,81,82,82,82
johto,82,80,82,NaN,81,82,82,82
työtov,81,79,81,81,NaN,81,81,81
työymp,82,80,82,82,81,NaN,82,82
palkkat,82,80,82,82,81,82,NaN,82
työteht,82,80,82,82,81,82,82,NaN


In [17]:
# korrelaatiokertoimien p-arvot; alle 0.05 suuruiset p-arvot tehostettu
ana.korre_p()

,ikä,palveluv,palkka,johto,työtov,työymp,palkkat,työteht
ikä,nan,0.000000,0.006762,0.279445,0.491341,0.363853,0.519682,0.091027
palveluv,0.000000,nan,0.029765,0.517476,0.862960,0.263491,0.968960,0.205402
palkka,0.006762,0.029765,nan,0.117673,0.296299,0.015071,0.003960,0.004894
johto,0.279445,0.517476,0.117673,nan,0.045059,0.000000,0.000000,0.000016
työtov,0.491341,0.862960,0.296299,0.045059,nan,0.009661,0.440759,0.005012
työymp,0.363853,0.263491,0.015071,0.000000,0.009661,nan,0.000000,0.000000
palkkat,0.519682,0.968960,0.003960,0.000000,0.440759,0.000000,nan,0.000005
työteht,0.091027,0.205402,0.004894,0.000016,0.005012,0.000000,0.000005,nan


## Lopuksi

Lisätietoa: https://tilastoapu.wordpress.com/python

Jos funktiot eivät toimi sinulle mieleisellä tavalla, niin voit vapaasti muuttaa ja täydentää niitä **ana.py**-moduulissa.

Jos haluat graafisia esityksiä, niin voit tehdä niitä Excelissä **ana_excel**-funktion laskemista tuloksista. Ensisijaisesti suosittelen kuitenkin seaborn-paketin käyttöä grafiikkaan: https://github.com/taanila/kaaviot/blob/master/kaaviot.ipynb.